In [1]:
import os
from datetime import datetime
import time
import threading
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_io as tf_io

2022-06-15 17:02:53.549643: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-15 17:02:53.549689: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print("tensorflow-io version: {}".format(tf_io.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.26.0
tensorflow version: 2.9.1


In [3]:
COLUMNS = [
          #  labels
           'class',
          #  low-level features
           'lepton_1_pT',
           'lepton_1_eta',
           'lepton_1_phi',
           'lepton_2_pT',
           'lepton_2_eta',
           'lepton_2_phi',
           'missing_energy_magnitude',
           'missing_energy_phi',
          #  high-level derived features
           'MET_rel',
           'axial_MET',
           'M_R',
           'M_TR_2',
           'R',
           'MT2',
           'S_R',
           'M_Delta_R',
           'dPhi_r_b',
           'cos(theta_r1)'
           ]

In [9]:
cwd = os.getcwd()
print(cwd)

/workspaces/python-mathematical-modelling-examples/supervised/notebooks


In [10]:
susy_iterator = pd.read_csv('/workspaces/python-mathematical-modelling-examples/SUSY.csv.gz', header=None, names=COLUMNS, chunksize=100000)


In [11]:
susy_df = next(susy_iterator)
susy_df.head()

,class,lepton_1_pT,lepton_1_eta,lepton_1_phi,lepton_2_pT,lepton_2_eta,lepton_2_phi,missing_energy_magnitude,missing_energy_phi,MET_rel,axial_MET,M_R,M_TR_2,R,MT2,S_R,M_Delta_R,dPhi_r_b,cos(theta_r1)
0,0.0,0.972861,0.653855,1.176225,1.157156,-1.739873,-0.874309,0.567765,-0.175000,0.810061,-0.252552,1.921887,0.889637,0.410772,1.145621,1.932632,0.994464,1.367815,0.040714
1,1.0,1.667973,0.064191,-1.225171,0.506102,-0.338939,1.672543,3.475464,-1.219136,0.012955,3.775174,1.045977,0.568051,0.481928,0.000000,0.448410,0.205356,1.321893,0.377584
2,1.0,0.444840,-0.134298,-0.709972,0.451719,-1.613871,-0.768661,1.219918,0.504026,1.831248,-0.431385,0.526283,0.941514,1.587535,2.024308,0.603498,1.562374,1.135454,0.180910
3,1.0,0.381256,-0.976145,0.693152,0.448959,0.891753,-0.677328,2.033060,1.533041,3.046260,-1.005285,0.569386,1.015211,1.582217,1.551914,0.761215,1.715464,1.492257,0.090719
4,1.0,1.309996,-0.690089,-0.676259,1.589283,-0.693326,0.622907,1.087562,-0.381742,0.589204,1.365479,1.179295,0.968218,0.728563,0.000000,1.083158,0.043429,1.154854,0.094859


In [12]:
# Number of datapoints and columns
len(susy_df), len(susy_df.columns)


(100000, 19)

In [13]:
# Number of datapoints belonging to each class (0: background noise, 1: signal)
len(susy_df[susy_df["class"]==0]), len(susy_df[susy_df["class"]==1])

(54025, 45975)

In [14]:
train_df, test_df = train_test_split(susy_df, test_size=0.4, shuffle=True)
print("Number of training samples: ",len(train_df))
print("Number of testing sample: ",len(test_df))

x_train_df = train_df.drop(["class"], axis=1)
y_train_df = train_df["class"]

x_test_df = test_df.drop(["class"], axis=1)
y_test_df = test_df["class"]

# The labels are set as the kafka message keys so as to store data
# in multiple-partitions. Thus, enabling efficient data retrieval
# using the consumer groups.
x_train = list(filter(None, x_train_df.to_csv(index=False).split("\n")[1:]))
y_train = list(filter(None, y_train_df.to_csv(index=False).split("\n")[1:]))

x_test = list(filter(None, x_test_df.to_csv(index=False).split("\n")[1:]))
y_test = list(filter(None, y_test_df.to_csv(index=False).split("\n")[1:]))

Number of training samples:  60000
Number of testing sample:  40000


In [15]:
NUM_COLUMNS = len(x_train_df.columns)
len(x_train), len(y_train), len(x_test), len(y_test)

(60000, 60000, 40000, 40000)

In [16]:
def error_callback(exc):
    raise Exception('Error while sendig data to kafka: {0}'.format(str(exc)))

def write_to_kafka(topic_name, items):
  count=0
  producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'])
  for message, key in items:
    producer.send(topic_name, key=key.encode('utf-8'), value=message.encode('utf-8')).add_errback(error_callback)
    count+=1
  producer.flush()
  print("Wrote {0} messages into topic: {1}".format(count, topic_name))

write_to_kafka("susy-train", zip(x_train, y_train))
write_to_kafka("susy-test", zip(x_test, y_test))

Wrote 60000 messages into topic: susy-train
Wrote 40000 messages into topic: susy-test


In [18]:
def decode_kafka_item(item):
  message = tf.io.decode_csv(item.message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(item.key)
  return (message, key)

BATCH_SIZE=64
SHUFFLE_BUFFER_SIZE=64
train_ds = tf_io.IODataset.from_kafka('susy-train', partition=0, offset=0)
train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
train_ds = train_ds.map(decode_kafka_item)
train_ds = train_ds.batch(BATCH_SIZE)

2022-06-15 17:51:01.139546: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available
2022-06-15 17:51:01.155378: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 AVX512F FMA
2022-06-15 17:51:02.811879: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-15 17:51:02.811921: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-15 17:51:02.827695: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-0d7959): /proc/driver/nvidia/version does not exist
2022-06-15 17:51:02.911475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAP

In [19]:
# Set the parameters

OPTIMIZER="adam"
LOSS=tf.keras.losses.BinaryCrossentropy(from_logits=True)
METRICS=['accuracy']
EPOCHS=10

In [20]:
# design/build the model
model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(NUM_COLUMNS,)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2432      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 1

In [21]:
# compile the model
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

In [22]:
# fit the model
model.fit(train_ds, epochs=EPOCHS)

Epoch 1/10


/opt/python/3.10.4/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2022-06-15 17:58:17.152475: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 0


     27/Unknown - 2s 9ms/step - loss: 0.6439 - accuracy: 0.6181 

2022-06-15 17:58:18.051833: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 1024
2022-06-15 17:58:18.262368: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 2048


     88/Unknown - 3s 12ms/step - loss: 0.5750 - accuracy: 0.6946

...

896/938 [===========================>..] - ETA: 1s - loss: 0.4435 - accuracy: 0.7949

2022-06-15 18:02:53.122624: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 58368


912/938 [============================>.] - ETA: 0s - loss: 0.4438 - accuracy: 0.7947

2022-06-15 18:02:53.626492: I tensorflow_io/core/kernels/kafka_kernels.cc:248] Kafka stream starts with current offset: 59392


938/938 [==============================] - 28s 29ms/step - loss: 0.4436 - accuracy: 0.7949


In [23]:
test_ds = tf_io.experimental.streaming.KafkaGroupIODataset(
    topics=["susy-test"],
    group_id="testcg",
    servers="127.0.0.1:9092",
    stream_timeout=10000,
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

def decode_kafka_test_item(raw_message, raw_key):
  message = tf.io.decode_csv(raw_message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(raw_key)
  return (message, key)

test_ds = test_ds.map(decode_kafka_test_item)
test_ds = test_ds.batch(BATCH_SIZE)

Instructions for updating:
Use `tf.data.Dataset.take_while(...)


2022-06-15 18:13:20.688979: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: session.timeout.ms=7000
2022-06-15 18:13:20.689056: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: max.poll.interval.ms=8000
2022-06-15 18:13:20.689084: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: auto.offset.reset=earliest
2022-06-15 18:13:20.689098: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: group.id=testcg
2022-06-15 18:13:20.689115: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: bootstrap.servers=127.0.0.1:9092
2022-06-15 18:13:20.689149: I tensorflow_io/core/kernels/kafka_kernels.cc:919] max num of messages per batch: 10000
2022-06-15 18:13:20.689161: I tensorflow_io/core/kernels/kafka_kernels.cc:938] Creating the kafka consumer
2022-06-15 18:13:20.689472: I tensorflow_io/core/kernels/kafka_kernels.cc:945] Subscribing to the kafka topic: susy-test


In [24]:
res = model.evaluate(test_ds)
print("test loss, test acc:", res)

2022-06-15 18:14:40.069674: E tensorflow_io/core/kernels/kafka_kernels.cc:774] REBALANCE: Local: Assign partitions
2022-06-15 18:14:40.073068: E tensorflow_io/core/kernels/kafka_kernels.cc:776] Retrieved committed offsets with status code: 0
2022-06-15 18:14:40.073116: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: susy-test[0], OFFSET: -1001 ERROR_CODE: 0
2022-06-15 18:14:40.073129: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: susy-test[1], OFFSET: -1001 ERROR_CODE: 0
2022-06-15 18:14:40.073137: I tensorflow_io/core/kernels/kafka_kernels.cc:802] REBALANCE: Assigning partitions


    332/Unknown - 1s 2ms/step - loss: 0.4389 - accuracy: 0.7951

2022-06-15 18:14:40.889830: I tensorflow_io/core/kernels/kafka_kernels.cc:996] EOF reached for all 2 partition(s)


625/625 [==============================] - 11s 17ms/step - loss: 0.4350 - accuracy: 0.7976
test loss, test acc: [0.4350031614303589, 0.7975749969482422]


2022-06-15 18:14:50.890318: E tensorflow_io/core/kernels/kafka_kernels.cc:1001] Local: Timed out


In [25]:
online_train_ds = tf_io.experimental.streaming.KafkaBatchIODataset(
    topics=["susy-train"],
    group_id="cgonline",
    servers="127.0.0.1:9092",
    stream_timeout=10000, # in milliseconds, to block indefinitely, set it to -1.
    configuration=[
        "session.timeout.ms=7000",
        "max.poll.interval.ms=8000",
        "auto.offset.reset=earliest"
    ],
)

2022-06-15 18:20:18.801901: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: session.timeout.ms=7000
2022-06-15 18:20:18.801949: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: max.poll.interval.ms=8000
2022-06-15 18:20:18.801965: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: auto.offset.reset=earliest
2022-06-15 18:20:18.801974: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: group.id=cgonline
2022-06-15 18:20:18.801984: I tensorflow_io/core/kernels/kafka_kernels.cc:879] Kafka configuration: bootstrap.servers=127.0.0.1:9092
2022-06-15 18:20:18.802001: I tensorflow_io/core/kernels/kafka_kernels.cc:919] max num of messages per batch: 10000
2022-06-15 18:20:18.802010: I tensorflow_io/core/kernels/kafka_kernels.cc:938] Creating the kafka consumer
2022-06-15 18:20:18.802341: I tensorflow_io/core/kernels/kafka_kernels.cc:945] Subscribing to the kafka topic: susy-train


In [26]:
def decode_kafka_online_item(raw_message, raw_key):
  message = tf.io.decode_csv(raw_message, [[0.0] for i in range(NUM_COLUMNS)])
  key = tf.strings.to_number(raw_key)
  return (message, key)

for mini_ds in online_train_ds:
  mini_ds = mini_ds.shuffle(buffer_size=32)
  mini_ds = mini_ds.map(decode_kafka_online_item)
  mini_ds = mini_ds.batch(32)
  if len(mini_ds) > 0:
    model.fit(mini_ds, epochs=3)

2022-06-15 18:21:06.187242: E tensorflow_io/core/kernels/kafka_kernels.cc:774] REBALANCE: Local: Assign partitions
2022-06-15 18:21:06.188239: E tensorflow_io/core/kernels/kafka_kernels.cc:776] Retrieved committed offsets with status code: 0
2022-06-15 18:21:06.188280: I tensorflow_io/core/kernels/kafka_kernels.cc:787] REBALANCE: susy-train[0], OFFSET: -1001 ERROR_CODE: 0
2022-06-15 18:21:06.188290: I tensorflow_io/core/kernels/kafka_kernels.cc:802] REBALANCE: Assigning partitions


Epoch 1/3
313/313 [==============================] - 1s 3ms/step - loss: 0.4455 - accuracy: 0.7944
Epoch 2/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4416 - accuracy: 0.7982
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4392 - accuracy: 0.7978
Epoch 1/3
313/313 [==============================] - 1s 3ms/step - loss: 0.4556 - accuracy: 0.7917
Epoch 2/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4486 - accuracy: 0.7951
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4476 - accuracy: 0.7914
Epoch 1/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4491 - accuracy: 0.7939
Epoch 2/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4443 - accuracy: 0.7961
Epoch 3/3
313/313 [==============================] - 1s 2ms/step - loss: 0.4414 - accuracy: 0.7967
Epoch 1/3
313/313 [==============================] - 1s 3ms/step - loss: 0.4458 - accuracy: 0.7947
Epoch 2/3


2022-06-15 18:21:20.898293: I tensorflow_io/core/kernels/kafka_kernels.cc:996] EOF reached for all 1 partition(s)
2022-06-15 18:21:30.918402: E tensorflow_io/core/kernels/kafka_kernels.cc:1001] Local: Timed out
